# Environment

In [4]:
from __future__ import print_function, division
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile

from scipy.stats import bernoulli
from scipy.stats import beta as bt
from scipy.stats import dirichlet
from scipy.special import digamma
from scipy.stats import loggamma 
from scipy.stats import expon
from scipy.cluster.vq import whiten

import time

import random

import wave

In [5]:
'''
data_paths = [["/Users/srivatsavpyda/Desktop/Unsupervised Learning/Homework_3/data/mixed_sound_dataset/dataset1/rsm2_mB.wav", 
        "/Users/srivatsavpyda/Desktop/Unsupervised Learning/Homework_3/data/mixed_sound_dataset/dataset1/rsm2_mA.wav"],
       ["/Users/srivatsavpyda/Desktop/Unsupervised Learning/Homework_3/data/mixed_sound_dataset/dataset2/rss_mB.wav",
       "/Users/srivatsavpyda/Desktop/Unsupervised Learning/Homework_3/data/mixed_sound_dataset/dataset2/rss_mA.wav"],
       ["/Users/srivatsavpyda/Desktop/Unsupervised Learning/Homework_3/data/mixed_sound_dataset/dataset3/rssd_B.wav",
       "/Users/srivatsavpyda/Desktop/Unsupervised Learning/Homework_3/data/mixed_sound_dataset/dataset3/rssd_A.wav"]]
'''


data_paths = [["C:\\Users\\Srivatsav\\Desktop\\Unsupervised_Learning\\uml-hw-3\\data\\mixed_sound_dataset\\dataset1\\rsm2_mB.wav", 
        "C:\\Users\\Srivatsav\\Desktop\\Unsupervised_Learning\\uml-hw-3\\data\\mixed_sound_dataset\\dataset1\\rsm2_mA.wav"],
       ["C:\\Users\\Srivatsav\\Desktop\\Unsupervised_Learning\\uml-hw-3\\data\\mixed_sound_dataset\\dataset2\\rss_mB.wav",
       "C:\\Users\\Srivatsav\\Desktop\\Unsupervised_Learning\\uml-hw-3\\data\\mixed_sound_dataset\\dataset2\\rss_mA.wav"],
       ["C:\\Users\\Srivatsav\\Desktop\\Unsupervised_Learning\\uml-hw-3\\data\\mixed_sound_dataset\\dataset3\\rssd_B.wav",
       "C:\\Users\\Srivatsav\\Desktop\\Unsupervised_Learning\\uml-hw-3\\data\\mixed_sound_dataset\\dataset3\\rssd_A.wav"]]


data_paths = np.array(data_paths)

# Functions

## Audio Write Functions

In [388]:
scipy.io.wavfile.write("testout.wav", rates[0], scipy.io.wavfile.read(data_paths[0,0])[1])

In [351]:
def write_to_audio(w, x, rates):
    
    for i in range(w.shape[0]):
        out = w[i,:].dot(x)
        print(np.max(out))
        print(np.min(out))
        st = "out%d.wav" % i
        scipy.io.wavfile.write(st, rates[i], out)

## Negentropy Functions

In [83]:
def g_1(u):
    a_1 = 1
    u_cosh = np.cosh(a_1 * u)
    u_log = np.log(u_cosh)
    final = (1/a_1) * u_log
    return final
def g_1_derivative(u):
    a_1 = 1
    return np.tanh(a_1 * u)

In [84]:
def g_2(u):
    return -1 * np.exp(-1 * (u * u) / 2)
def g_2_derivative(u):
    return u * np.exp(-1 * (u * u) / 2)

## Update Functions

In [218]:
def g_1_expectation(w, x, t, sample_size):
    indices = random.sample(range(t), sample_size)
    #print("wdotx")
    #print(w.dot(x[:,indices]))
    x_analyzed = x[:,indices]
    g_1_value = g_1(w.dot(x[:,indices]))
    new_indices = np.isfinite(g_1_value)
    g_1_value = g_1_value[new_indices]
    x_analyzed = x_analyzed[:,new_indices]
    
    values = x_analyzed * g_1_value
    #values = values[:, np.isfinite(values).all(axis=0)]
    #print("g1val")
    #print(g_1_value[:10])
    #print("xval")
    #print(x_analyzed[:,:10])
    #print("vals")
    #print(values[:,:10])
    #values_sum = np.sum(values, axis=1)
    #values_mean = values_sum / sample_size
    values_mean = np.mean(values, axis=1, dtype=np.float64)
    #print("meanval")
    #print(values_mean)
    #print(values_mean)
    return values_mean

In [145]:
a = np.array([[ 5, 1 ,3], 
                  [ 1, 1 ,1]])
b = np.array([1, 2])
print(b[np.newaxis,:].T)
#print(a / np.linalg.norm(a, axis=1)[np.newaxis,:].T)
#print(a[0,1] / np.linalg.norm(a, axis=1)[np.newaxis,:].T[1,0])
#print(b.dot(a))
#print(b[np.newaxis, :].T - b[np.newaxis, :].T * a)

[[1]
 [2]]


In [212]:
def g_1_derivative_expectation(w, x, t, sample_size):
    indices = random.sample(range(t), sample_size)
    w_dot_x = w.dot(x[:,indices])
    values = g_1_derivative(w_dot_x)
    #final_value = np.mean(values, axis=1)
    final_value = np.mean(values)
    #print(final_value)
    return final_value

In [147]:
def normalizer(w):
    w = w / np.sqrt(np.linalg.norm(np.dot(w, w.T)))
    return w

In [148]:
def decorellater(w, threshold):
    w_old = 0
    w_new = w
    while(np.linalg.norm(w_new - w_old) > threshold):
        w_old = w_new
        w_new = 3/2 * w - 1/2 * (w.dot(w.T.dot(w)))
        
    return w_new

In [149]:
def iterative_normalizer(w):
    for i in range(w.shape[0]):
        total = np.zeros((w.shape[1]))
        cur_row = w[i,:]
        for j in range(i):
            iter_row = w[j,:]
            it = np.outer(cur_row, iter_row)
            print(it)
            it = it.dot(iter_row)
            print(it)
            total = total + it
            #total = total + cur_row[np.newaxis,:].T.dot(w[j,:]).dot(w[j,:])

        w[i,:] = w[i,:] - total
        w[i,:] = w[i,:]/np.sqrt(w[i,:].T.dot(w[i,:]))
    return w

In [246]:
def single_update(w, x, t, sample_size):
    g_exp = g_1_expectation(w, x, t, sample_size)
    g_der_exp = g_1_derivative_expectation(w, x, t, sample_size)
    return g_exp - (g_der_exp * w)

In [359]:
def update_w(i, w, x, t, sample_size):
    #for i in range(w.shape[0]):
    w[i,:] = single_update(w[i,:], x, t, sample_size)
    total = np.zeros((w.shape[1]))
    #print(w)

    for j in range(i):
        it = np.outer(w[i,:], w[j,:])
        it = it.dot(w[j,:])
        total = total + it
    w[i,:] = w[i,:] - total

    #print(w)
    w[i,:] = w[i,:]/np.linalg.norm(w[i,:])
    #print(w)
        
    
    
    '''
    print("w")
    print(w)
    g_exp = g_1_expectation(w, x, t, sample_size)
    g_der_exp = g_1_derivative_expectation(w, x, t, sample_size)
    print("exp")
    print(g_der_exp)
    w_new = g_exp[np.newaxis,:].T - g_der_exp[np.newaxis,:].T * w
    print("prenorm")
    print(w_new)
    #w_new = w_new / np.linalg.norm(w_new, axis=1)[np.newaxis,:].T
    w_new = normalizer(w_new)
    print("postnorm")
    print(w_new)
    w_new = decorellater(w_new, 0.01)
    print("postdecor")
    print(w_new)
    #w_new = iterative_normalizer(w_new)
    '''
    return w

# Convergence Measures

In [358]:
def w_has_converged(i, w, w_new, threshold):
    s = w[i,:].dot(w_new[i,:])
    s_1 = np.absolute(1-s)
    if s_1 < threshold:
        return s_1, s, True
    return s_1, s, False
    
    #for i in range(w.shape[0]):
    #    s = s + w[i,:].dot(w_new[i,:])
    #sum_altered = np.absolute(1-s)
    #if sum_altered <= threshold:
    #    return sum_altered, s, True
    #return sum_altered, s, False

# Data Preparation

## Hyperparameters

In [378]:
sample_size = 10000
n = 2
d = 2
threshold = 0.01

## Data Loading

In [379]:
def center(z):
    return z / np.mean(z)

In [380]:
current_sample = 0
rates = []
data = []
for i in range(n):
    rate, dat = scipy.io.wavfile.read(data_paths[current_sample,i])
    rates.append(rate)
    
    dat = center(dat)
    data.append(dat)

## Variables

In [381]:
t = data[0].shape[0]
x = np.zeros((n, t))
                                              
for i in range(n):
    x[i,:] = data[i]
    
x = whiten(x)
    
w = np.random.rand(n, d)

In [382]:
#w_new = update_w(w, x, t, sample_size)

In [383]:
#w

In [384]:
#w_new

### Data Visualization

In [385]:
print("here")
time = 10

#plt.figure(figsize=[8,6], dpi=150)
#plt.plot(range(time), x[0, :time])
#plt.ylabel("Signal")
#plt.xlabel("Time")
#plt.title("Signal Over Time")
#plt.show()

here


# Iterative Loop

In [386]:
w_record = []
dot_product_record = []
loss_record = []

In [387]:
for i in range(n):
    j=1
    while(True):
        if j % 10 == 0:
            print("Iteration %d is starting" % j)
            print(w)

        w_new = update_w(i, w, x, t, sample_size)
        #print(w_new)

        loss, dot_product, has_converged = w_has_converged(i, w, w_new, threshold)

        if has_converged:
            print("here")
            print(w)
            print(w_new)
            break
            
        w = w_new
        w_record.append(w)
        loss_record.append(loss)
        dot_product_record.append(dot_product)
        j = j + 1

here
[[ 0.7036251   0.7105714 ]
 [ 0.27137632  0.4337221 ]]
[[ 0.7036251   0.7105714 ]
 [ 0.27137632  0.4337221 ]]
here
[[ 0.7036251   0.7105714 ]
 [-0.70710678 -0.70710678]]
[[ 0.7036251   0.7105714 ]
 [-0.70710678 -0.70710678]]


# Evaluation

In [352]:
w = np.array([[-0.70986198, -0.7043408 ]
 ,[ 0.89442719,  0.4472136 ]])
write_to_audio(w, x, rates)

801144.021573
-365514.816133
346760.003986
-760038.191305


In [ ]:
plt.figure(figsize=[8,6], dpi=150)
plt.plot(range(len(dot_product_record)), dot_product_record)
plt.ylabel("Signal")
plt.xlabel("Time")
plt.title("Signal Over Time")
plt.show()

In [ ]:
print(dot_product_record[:100])

In [57]:
x = np.array([[1], [2]])
y = np.array([[3, 4]])
print(x.shape)
print(y.shape)

(2L, 1L)
(1L, 2L)


In [56]:
np.dot(np.array([[1], [2]]), np.array([[3, 4]]))

array([[3, 4],
       [6, 8]])

In [45]:
x = 

array([[0, 1],
       [5, 2]])

In [46]:
x.T

array([[0, 5],
       [1, 2]])

In [38]:
np.transpose(x)

array([0, 1])

In [26]:
np.log(0)

-inf

In [185]:
a = np.array([[2,3], [4,5]])

In [186]:
b = np.array([1, 2])

In [187]:
print(a)

[[2 3]
 [4 5]]


In [188]:
print(b)

[1 2]


In [189]:
a * b

array([[ 2,  6],
       [ 4, 10]])

In [190]:
a.dot(b)

array([ 8, 14])